In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, accuracy_score,recall_score,precision_score,f1_score
import xgboost as xgb
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from tpot import TPOTClassifier

In [2]:
df_default = pd.read_excel("default_of_credit_card_clients.xls")

In [3]:
df_default

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [4]:
df_default.columns

Index(['Unnamed: 0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9',
       'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19',
       'X20', 'X21', 'X22', 'X23', 'Y'],
      dtype='object')

In [5]:
columns = df_default.iloc[0]
val = list(columns.values)
df_default.set_axis(val, axis=1, inplace=True)
df_default.drop(0, inplace = True)

In [6]:
df_default

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [7]:
#Pre-processing
#df_default.drop("ID", axis=1, inplace=True)
#Creation of a copied dataframe
#df_default_new = df_default.copy()
#df_default_new.drop("ID", axis=1, inplace=True)
#col = df_default.columns
#categorical_columns = ["SEX", "EDUCATION", "MARRIAGE", 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
#hot_encoded_df = pd.get_dummies(df_default, columns= categorical_columns)
#print(hot_encoded_df)
#df_default_new = pd.DataFrame(data=hot_encoded_df)
#df_default_new.drop(categorical_columns, axis=1, inplace=True)
#df_default_new = pd.concat([df_default_new, hot_encoded_df], axis=1)

#Creation of dummy variable by hand
#Creation of dummy variables
L = ["SEX", "EDUCATION", "MARRIAGE", 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
for column in L:  
    for type in df_default[f"{column}"].unique():
        df_default[f"{column}_{type}"] = np.where(df_default[f"{column}"] == type,1,0)

# Age
for i in range(15,86,5):
    df_default[f"AGE_{i}_{i+5}"] = np.where((df_default["AGE"] >= i) & (df_default["AGE"] < i+5), 1,0)

#Bill column
for i in range(1,7):
    df_default[f"BILL_AMT{i}"] = (df_default[f"BILL_AMT{i}"] - df_default[f"BILL_AMT{i}"].mean())/df_default[f"BILL_AMT{i}"].std()

#Pay column
for i in range(1,7):
    df_default[f"BILL_AMT{i}"] = (df_default[f"PAY_AMT{i}"] - df_default[f"PAY_AMT{i}"].mean())/df_default[f"PAY_AMT{i}"].std()


In [8]:
df_default

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,AGE_40_45,AGE_45_50,AGE_50_55,AGE_55_60,AGE_60_65,AGE_65_70,AGE_70_75,AGE_75_80,AGE_80_85,AGE_85_90
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,0,0,0,0,0,0
2,2,120000,2,2,2,26,-1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,90000,2,2,2,34,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,50000,2,2,1,37,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,50000,1,2,1,57,-1,0,-1,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,29996,220000,1,3,1,39,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29997,29997,150000,1,3,2,43,-1,-1,-1,-1,...,1,0,0,0,0,0,0,0,0,0
29998,29998,30000,1,2,2,37,4,3,2,-1,...,0,0,0,0,0,0,0,0,0,0
29999,29999,80000,1,3,1,41,1,-1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [9]:
df_default.drop(columns=["ID","SEX", "EDUCATION", "MARRIAGE", 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'])


,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,...,AGE_40_45,AGE_45_50,AGE_50_55,AGE_55_60,AGE_60_65,AGE_65_70,AGE_70_75,AGE_75_80,AGE_80_85,AGE_85_90
1,20000,24,-0.341936,-0.227082,-0.296796,-0.308057,-0.314131,-0.293377,0,689,...,0,0,0,0,0,0,0,0,0,0
2,120000,26,-0.341936,-0.213584,-0.240001,-0.244226,-0.314131,-0.180875,0,1000,...,0,0,0,0,0,0,0,0,0,0
3,90000,34,-0.250287,-0.191884,-0.240001,-0.244226,-0.248679,-0.012122,1518,1500,...,0,0,0,0,0,0,0,0,0,0
4,50000,37,-0.221187,-0.169358,-0.228641,-0.237842,-0.244162,-0.237126,2000,2019,...,0,0,0,0,0,0,0,0,0,0
5,50000,57,-0.221187,1.335012,0.271161,0.266429,-0.269034,-0.255183,2000,36681,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,220000,39,0.171247,0.611038,-0.012647,-0.113562,0.013131,-0.237126,8500,20000,...,0,0,0,0,0,0,0,0,0,0
29997,150000,43,-0.231028,-0.103953,0.214252,-0.299823,-0.314131,-0.293377,1837,3526,...,1,0,0,0,0,0,0,0,0,0
29998,30000,37,-0.341936,-0.256985,0.952709,-0.039964,-0.183226,-0.118999,0,0,...,0,0,0,0,0,0,0,0,0,0
29999,80000,41,4.844235,-0.109031,-0.229891,-0.185117,3.152484,-0.1919,85900,3409,...,1,0,0,0,0,0,0,0,0,0


In [10]:
#columns = df_default.iloc[0]
val = list(columns.values)
#df_default.set_axis(val, axis=1, inplace=True)

In [11]:
#df_default.drop(0, inplace = True)

In [12]:
labels = np.array(df_default['default payment next month'])
labels = labels.astype('int')
features = df_default.drop('default payment next month', axis = 1)
features_list = val
features = np.array(features)

In [13]:
print(val[1:-1])

['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']


In [14]:
x = df_default[val[1:-1]]
y = df_default['default payment next month']
train_features, test_features, train_label, test_label = train_test_split(features, labels, test_size = 0.33, random_state = 42)

In [26]:
rf = RandomForestClassifier()
    #rf.fit(train_features, train_label)

grid_values = { 
'n_estimators': [200, 500],
'max_features': ['auto', 'sqrt', 'log2'],
'max_depth' : [4,5,6,7,8],
'criterion' :['gini', 'entropy']
}
grid_clf_acc = GridSearchCV(rf, param_grid = grid_values,scoring = 'recall', cv=5)
grid_clf_acc.fit(train_features, train_label)
#[0.001,0.009,0.01,0.09,1,5,10,25]
#['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start']
y_pred = grid_clf_acc.predict(test_features)

print('Accuracy Score : ' + str(accuracy_score(test_label,y_pred)))
print('Precision Score : ' + str(precision_score(test_label,y_pred)))
print('Recall Score : ' + str(recall_score(test_label,y_pred)))
print('F1 Score : ' + str(f1_score(test_label,y_pred)))

#Confusion matrix
print('Confusion Matrix : \n' + str(confusion_matrix(test_label,y_pred)))

c:\miniconda\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\miniconda\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\miniconda\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTre

KeyboardInterrupt: 

In [15]:
rf = RandomForestClassifier()
rf.fit(train_features, train_label)

RandomForestClassifier()

In [16]:
prediction = rf.predict(test_features)
errors = abs(prediction - test_label)
print(round(np.mean(errors), 2))

0.18


In [25]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(features):   
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    rf = RandomForestClassifier()
    #rf.fit(X_train, y_train)

    grid_values = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
    }
    grid_clf_acc = GridSearchCV(rf, param_grid = grid_values,scoring = 'recall')
    grid_clf_acc.fit(X_train, y_train)
    #[0.001,0.009,0.01,0.09,1,5,10,25]
    #['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start']
    y_pred = grid_clf_acc.predict(X_test)

    #print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
    #print('Precision Score : ' + str(precision_score(y_test,y_pred)))
    #print('Recall Score : ' + str(recall_score(y_test,y_pred)))
    #print('F1 Score : ' + str(f1_score(y_test,y_pred)))

    #Confusion matrix
    #print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))
    
    scores.append(mean_squared_error(y_test, y_pred))

scores

c:\miniconda\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\miniconda\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\miniconda\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTre

KeyboardInterrupt: 

In [23]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(features):   
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    model = RandomForestClassifier(random_state= 101).fit(X_train,y_train)

    grid_search = {
    'criterion': [model.best_params_['criterion']],
    'max_depth': [model.best_params_['max_depth']],
    'max_features': [model.best_params_['max_features']],
    'min_samples_leaf': [model.best_params_['min_samples_leaf'] - 2, 
                         model.best_params_['min_samples_leaf'], 
                         model.best_params_['min_samples_leaf'] + 2],
    'min_samples_split': [model.best_params_['min_samples_split'] - 3, 
                          model.best_params_['min_samples_split'], 
                          model.best_params_['min_samples_split'] + 3],
    'n_estimators': [model.best_params_['n_estimators'] - 150, 
                     model.best_params_['n_estimators'] - 100, 
                     model.best_params_['n_estimators'], 
                     model.best_params_['n_estimators'] + 100, 
                     model.best_params_['n_estimators'] + 150]
    }

    clf = RandomForestClassifier()
    model = GridSearchCV(estimator = clf, param_grid = grid_search, 
                                cv = 4, verbose= 5, n_jobs = -1)
    model.fit(X_train,y_train)

    predictionforest = model.best_estimator_.predict(X_test)
    print(confusion_matrix(y_test,predictionforest))
    print(classification_report(y_test,predictionforest))
    acc4 = accuracy_score(y_test,predictionforest)

    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)

    print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
    print('Precision Score : ' + str(precision_score(y_test,y_pred)))
    print('Recall Score : ' + str(recall_score(y_test,y_pred)))
    print('F1 Score : ' + str(f1_score(y_test,y_pred)))

    #Confusion matrix
    print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))
    
    y_pred = rf.predict(X_test)

AttributeError: 'RandomForestClassifier' object has no attribute 'best_params_'

In [319]:
count = 0
for x, y in zip(prediction, test_label):
    if x == y:
        count += 1
count/len(prediction)

0.8171717171717172

In [320]:
mean_squared_error(test_label, prediction)

0.18282828282828284

In [321]:
#Try with XG_boost
xgb_model = xgb.XGBRegressor(objective='reg:linear', random_state = 42)
xgb_model.fit(train_features, train_label)
prediction_xgb = xgb_model.predict(test_features) 

[13:22:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [322]:
mse = mean_squared_error(test_label, prediction_xgb)
mse

0.14322233560694073

In [323]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(features):   
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    xgb_model = xgb.XGBRegressor(objective="reg:linear")
    xgb_model.fit(X_train, y_train)
    
    y_pred = xgb_model.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred))

scores

[13:22:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[13:22:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[13:22:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[13:22:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[13:22:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscal

[0.1401127347645314,
 0.14108179869358167,
 0.13573327897763088,
 0.1455405273262789,
 0.14252107526388286]

In [ ]:
estimator = xgb.XGBRegressor(
    objective= 'reg:linear',
    nthread=4,
    seed=42
)

parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

grid_search.fit(x, y)
grid_search.best_estimator_

y_pred = xgb_model.predict(X_test)
    
scores.append(mean_squared_error(y_test, y_pred))

In [324]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(features):   
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    clf = svm.SVC()
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred))

scores

[0.21883333333333332,
 0.22166666666666668,
 0.212,
 0.23166666666666666,
 0.22183333333333333]

In [325]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(features):   
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred))

scores

[0.21883333333333332,
 0.22166666666666668,
 0.212,
 0.23166666666666666,
 0.22183333333333333]

In [331]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(features):   
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    clf = MLPClassifier(solver='sgd', alpha=1e-5,hidden_layer_sizes=(10, 10), learning_rate='adaptive', random_state=1)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred))
scores

[0.21883333333333332,
 0.22166666666666668,
 0.212,
 0.23166666666666666,
 0.22183333333333333]